## 1. 기본 LLM 체인 (Prompt + LLM)

In [1]:
from langchain_openai import ChatOpenAI

In [2]:
# model 
llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
# chain 실행
llm.invoke("지구의 자전 주기는?")

AIMessage(content='지구의 자전 주기는 약 24시간입니다. 이것은 하루의 길이를 결정하는 주기로, 지구가 자전하는 동안 해와 달이 번갈아 가면서 지구의 표면에 비추는 것을 의미합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 16, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7d23152a-efc4-4f94-b155-89e271f6f4bf-0', usage_metadata={'input_tokens': 16, 'output_tokens': 76, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in astronomy. Answer the question. <Question>: {input}'), additional_kwargs={})])

In [6]:
chain = prompt | llm

In [7]:
chain.invoke({"input": "지구의 자전 주기는?"})

AIMessage(content='지구의 자전 주기는 약 23시간 56분 4.1초 입니다. 이것은 지구가 자전하는데 걸리는 시간을 말하며, 하루의 길이를 결정하는 중요한 요소입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 30, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-540263c3-2e97-46bc-b6f5-434df1c2bd66-0', usage_metadata={'input_tokens': 30, 'output_tokens': 70, 'total_tokens': 100, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [3]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()


In [ ]:
chain = prompt | llm | output_parser

chain.invoke({"input": "지구의 자전 주기는?"})

## 2. 멀티 체인 (Multi-Chain)

In [10]:
# 단어가 들어왔을 때 영어로 바꿔주는 녀석
prompt1 = ChatPromptTemplate.from_template("translates {korean_word} to English.")

# 바꾼 영어를 영국 영어사전을 통해 한국말로 소개시켜주는 녀석
prompt2 = ChatPromptTemplate.from_template(
    "explain {english_word} using oxford dictionary to me in Korean."
)

llm = ChatOpenAI(model="gpt-4o-mini")

chain1 = prompt1 | llm | StrOutputParser()

chain1.invoke({"korean_word":"미래"})

'The Korean word "미래" translates to "future" in English.'

In [11]:
chain2 = (
    {"english_word": chain1}
    | prompt2
    | llm
    | StrOutputParser()
)

chain2.invoke({"korean_word":"미래"})

'"미래"라는 한국어 단어는 영어로 "future"로 번역됩니다. 옥스포드 사전에서 "future"는 다음과 같이 정의됩니다: \n\n1. **명사**: 아직 오지 않은 시간, 특히 현재 이후의 시간.\n2. **형용사**: 앞으로 일어날 사건이나 상황과 관련된.\n\n즉, "미래"는 현재 시점 이후에 발생할 가능성이 있는 모든 일을 포함하는 개념입니다.'

## 3. 체인을 실행하는 방법

- Runnable 프로토콜 사용
- 일관된 인터페이스를 사용하여 다양한 타입의 컴포넌트를 조합할 수 있음

invoke - 기본   
batch - 여러 개 한번에 처리할 때   
stream - 결과의 조각들을 스트리밍 -> 애니메이션이 나오는 건 이것 때문임.

In [6]:
prompt = ChatPromptTemplate.from_template("지구과학에서 {topic}에 대해 간단히 설명해주세요.")

chain = prompt | llm | output_parser

In [13]:
# 배치 메서드
topics = ["지구 공전", "화산 활동", "대륙 이동"]
results = chain.batch([{"topic": t} for t in topics])

for topic, result in zip(topics, results):
    print(f"{topic} 설명: {result[:50]}...")


지구 공전 설명: 지구 공전은 지구가 태양 주위를 한 바퀴 도는 과정을 말합니다. 이 과정은 약 365.25...
화산 활동 설명: 화산 활동은 지구 내부의 마그마가 지표로 분출되는 현상을 말합니다. 이 과정은 주로 지구의...
대륙 이동 설명: 대륙 이동 이론은 지구의 표면이 여러 개의 대륙판으로 나뉘어 있으며, 이들이 천천히 이동한...


In [7]:
# stream 메소드 사용
stream = chain.stream({"topic": "지진"})
print("stream 결과:")
for chunk in stream:
    print(chunk, end="", flush=True)
print()

stream 결과:
지진은 지구 내부의 에너지가 갑작스럽게 방출되면서 발생하는 자연현상입니다. 주로 지각의 단층선에서 발생하며, 지각의 판들이 서로 충돌하거나 분리되거나 미끄러질 때 생기는 긴장이 해소되면서 발생합니다. 이때 발생하는 에너지가 지진파로 전파되어 지표면에 도달하게 되며, 이를 통해 진동이 발생하게 됩니다.

지진의 크기는 리히터 규모 또는 모멘트 규모로 측정되며, 진동의 강도는 진도계로 측정됩니다. 지진은 규모와 진도에 따라 피해를 줄 수 있으며, 큰 지진은 쓰나미를 발생시키기도 합니다. 따라서 지진은 자연재해 중 하나로, 이에 대한 연구와 대비가 중요합니다.
